
# Example of High Gamma Filter

Below is a code sample for extracting high gamma power from a raw data file, followed by permutation cluster stats on that high gamma power data


In [7]:
import ieeg.viz.utils
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.calc import stats, scaling
from ieeg.process import parallelize
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
from misc_functions import calculate_RTs


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from bids import BIDSLayout
import mne
import os
import numpy as np

from misc_functions import calculate_RTs, save_sig_chans
import pickle

In [10]:
def relabel_axes(old_min, old_max, new_min, new_max):
    scale = (new_max - new_min) / (old_max - old_min)

    def format_func(value, tick_number):
        return new_min + value * scale

    plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(format_func))

### grab the data and set up variables

In [ ]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())

# raw = raw_from_layout(layout, subject=sub,
#                         extension='.edf', preload=True)
subjects = layout.get(return_type="id", target="subject")
for sub in subjects:

    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                            extension='.edf', desc='clean', preload=False) #get line-noise filtered data

    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)


    # do filtering now
    ## Crop raw data to minimize processing time
    good = crop_empty_data(filt)

    ## remove bad channels
    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    good.drop_channels(good.info['bads'])
    good.load_data()

    # CAR
    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)


    # make baseline 
    # make stimulus baseline EpochsTFR
    times=[-1, 0.5]
    trials = trial_ieeg(good, "Stimulus", times, preload=True)
    outliers_to_nan(trials, outliers=10) #this removes trial outliers BUT breaks the shape because different stimulus epochs will then have different trial numbers
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    data_dict = {}  # make something to store all the outputs

    for event, t in zip(("Stimulus", "Response"), ((-1, 1.5), (-1, 1.5))):
        times = [None, None]
        times[0] = t[0] - 0.5
        times[1] = t[1] + 0.5
        trials = trial_ieeg(good, event, times, preload=True)
        outliers_to_nan(trials, outliers=10)
        HG_ev1 = gamma.extract(trials, copy=False, n_jobs=1)
        crop_pad(HG_ev1, "0.5s")

        HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')
        # HG_ev1.resample(100)
        # HG_ev1.filenames = good.filenames

        # Store the variable in the dictionary with the desired name
        data_dict[f"HG_ev1_{event}"] = HG_ev1
        data_dict[f"HG_ev1_{event}_rescaled"] = HG_ev1_rescaled
        
    resp_evoke = data_dict["HG_ev1_Response"].average(method=lambda x: np.nanmean(x, axis=0))
    resp_evoke_rescaled = data_dict["HG_ev1_Response_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))
    stim_evoke = data_dict["HG_ev1_Stimulus"].average(method=lambda x: np.nanmean(x, axis=0))
    stim_evoke_rescaled = data_dict["HG_ev1_Stimulus_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))

    RTs, skipped = calculate_RTs(good)
    avg_RT = np.median(RTs)

    # Plot the evoked data
    fig = resp_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))  # 1 because z-score is unit-less and requires no scaling
    fig.savefig(save_dir + '_HG_ev1_Response_zscore_rt.png')

    fig = stim_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))  # 1 because z-score is unit-less and requires no scaling
    # Add a vertical line indicating the average reaction time to the plot
    for ax in fig.axes:
        ax.axvline(x=avg_RT, color='r', linestyle='--')
        
    fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore_rt.png')

    # decimate for stats
    resp = data_dict["HG_ev1_Response"] #i think it shouldn't be the rescaled data because that is already divided by baseline, right? So it shouldn't be significantly different than baseline.
    stim = data_dict["HG_ev1_Stimulus"]
    HG_base.decimate(2)
    resp.decimate(2)
    stim.decimate(2)

    # do permutation cluster
    resp_mask = stats.time_perm_cluster(resp._data, HG_base._data, 0.05, axis=0,
                                n_perm=1000, n_jobs=6, ignore_adjacency=1)
    stim_mask = stats.time_perm_cluster(stim._data, HG_base._data, 0.05, axis=0,
                                n_perm=1000, n_jobs=6, ignore_adjacency=1)
    
    #untested code 8/1
    # Assuming you have the channel names in a variable called 'channels'
    channels = good.ch_names

    # Save the significant channels for the response
    save_sig_chans(resp_mask, channels, sub, save_dir)

    # Save the significant channels for the stimulus
    save_sig_chans(stim_mask, channels, sub, save_dir)


    # plot stats
    plt.figure(figsize=(16, 8))  # Adjust the width and height as needed
    plt.imshow(stim_mask, aspect='auto')  # hold on from matlab
    relabel_axes(0, 2500, -1000, 1500)
    plt.savefig(save_dir + '_stimulus_stats_big.png', dpi=300)

    plt.figure(figsize=(16, 8))
    plt.imshow(resp_mask, aspect='auto')
    relabel_axes(0, 2500, -1000, 1500)
    plt.savefig(save_dir + '_response_stats_big.png', dpi=300)



### testing with single subject with prestimulus baseline period

In [11]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())

# raw = raw_from_layout(layout, subject=sub,
#                         extension='.edf', preload=True)
subjects = layout.get(return_type="id", target="subject")

sub='D0071'

filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False) #get line-noise filtered data

save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)


# do filtering now
## Crop raw data to minimize processing time
good = crop_empty_data(filt)

## remove bad channels
good.info['bads'] = channel_outlier_marker(good, 3, 2)
good.drop_channels(good.info['bads'])
good.load_data()

# CAR
ch_type = filt.get_channel_types(only_data_chs=True)[0]
good.set_eeg_reference(ref_channels="average", ch_type=ch_type)


# make baseline 
# make stimulus baseline EpochsTFR
times=[-1, 0.5]
trials = trial_ieeg(good, "Stimulus", times, preload=True)
outliers_to_nan(trials, outliers=10) #this removes trial outliers BUT breaks the shape because different stimulus epochs will then have different trial numbers
HG_base = gamma.extract(trials, copy=False, n_jobs=1)
crop_pad(HG_base, "0.5s")

data_dict = {}  # make something to store all the outputs

for event, t in zip(("Stimulus", "Response"), ((-1, 1.5), (-1, 1.5))):
    times = [None, None]
    times[0] = t[0] - 0.5
    times[1] = t[1] + 0.5
    trials = trial_ieeg(good, event, times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_ev1 = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_ev1, "0.5s")

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')
    # HG_ev1.resample(100)
    # HG_ev1.filenames = good.filenames

    # Store the variable in the dictionary with the desired name
    data_dict[f"HG_ev1_{event}"] = HG_ev1
    data_dict[f"HG_ev1_{event}_rescaled"] = HG_ev1_rescaled
    
resp_evoke = data_dict["HG_ev1_Response"].average(method=lambda x: np.nanmean(x, axis=0))
resp_evoke_rescaled = data_dict["HG_ev1_Response_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))
stim_evoke = data_dict["HG_ev1_Stimulus"].average(method=lambda x: np.nanmean(x, axis=0))
stim_evoke_rescaled = data_dict["HG_ev1_Stimulus_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))

RTs, skipped = calculate_RTs(good)
avg_RT = np.median(RTs)

# Plot the evoked data
fig = resp_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))  # 1 because z-score is unit-less and requires no scaling
fig.savefig(save_dir + '_HG_ev1_Response_zscore_rt.png')

fig = stim_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))  # 1 because z-score is unit-less and requires no scaling
# Add a vertical line indicating the average reaction time to the plot
for ax in fig.axes:
    ax.axvline(x=avg_RT, color='r', linestyle='--')
    
fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore_rt.png')

# decimate for stats
resp = data_dict["HG_ev1_Response"] #i think it shouldn't be the rescaled data because that is already divided by baseline, right? So it shouldn't be significantly different than baseline.
stim = data_dict["HG_ev1_Stimulus"]
HG_base.decimate(2)
resp.decimate(2)
stim.decimate(2)

# do permutation cluster
resp_mask = stats.time_perm_cluster(resp._data, HG_base._data, 0.05, axis=0,
                            n_perm=1000, n_jobs=6, ignore_adjacency=1)
stim_mask = stats.time_perm_cluster(stim._data, HG_base._data, 0.05, axis=0,
                            n_perm=1000, n_jobs=6, ignore_adjacency=1)

#untested code 8/1
# Assuming you have the channel names in a variable called 'channels'
channels = good.ch_names

# Save the significant channels for the response
save_sig_chans('Response', resp_mask, channels, sub, save_dir)

# Save the significant channels for the stimulus
save_sig_chans('Stimulus', stim_mask, channels, sub, save_dir)


# plot stats
plt.figure(figsize=(16, 8))  # Adjust the width and height as needed
plt.imshow(stim_mask, aspect='auto')  # hold on from matlab
relabel_axes(0, 2500, -1000, 1500)
plt.savefig(save_dir + '_stimulus_stats_big.png', dpi=300)

plt.figure(figsize=(16, 8))
plt.imshow(resp_mask, aspect='auto')
relabel_axes(0, 2500, -1000, 1500)
plt.savefig(save_dir + '_response_stats_big.png', dpi=300)



{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24})
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0069\ieeg\sub-D0069_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0069\ieeg\sub-D0069_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0069\ieeg\sub-D0069_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0069\ieeg\sub-D0069_acq-01_space-ACPC_electrodes.t

c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (133) does not match the number of channels in the raw data file (132). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0069\ieeg\sub-D0069_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0069\ieeg\sub-D0069_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0069\ieeg\sub-D0069_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0069\ieeg\sub-D0069_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Omitted 224 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (133) does not match the number of channels in the raw data file (132). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0069\ieeg\sub-D0069_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0069\ieeg\sub-D0069_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0069\ieeg\sub-D0069_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0069\ieeg\sub-D0069_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

Applying baseline correction (mode: zscore)
Used Annotations descriptions: ['Response/c25/n75', 'Response/c25/r25', 'Response/c25/r75', 'Response/c25/s25', 'Response/c25/s75', 'Response/c75/r25', 'Response/c75/r75', 'Response/c75/s25', 'Response/c75/s75', 'Response/i25/n25', 'Response/i25/r25', 'Response/i25/r75', 'Response/i25/s25', 'Response/i25/s75', 'Response/i75/n25', 'Response/i75/n75', 'Response/i75/r25', 'Response/i75/r75', 'Response/i75/s25', 'Response/i75/s75', 'Stimulus/c25/n75', 'Stimulus/c25/r25', 'Stimulus/c25/r75', 'Stimulus/c25/s25', 'Stimulus/c25/s75', 'Stimulus/c75/r25', 'Stimulus/c75/r75', 'Stimulus/c75/s25', 'Stimulus/c75/s75', 'Stimulus/i25/n25', 'Stimulus/i25/r25', 'Stimulus/i25/r75', 'Stimulus/i25/s25', 'Stimulus/i25/s75', 'Stimulus/i75/n25', 'Stimulus/i75/n75', 'Stimulus/i75/r25', 'Stimulus/i75/r75', 'Stimulus/i75/s25', 'Stimulus/i75/s75']
event_ids: {'c25/n75': 1, 'c25/r25': 2, 'c25/r75': 3, 'c25/s25': 4, 'c25/s75': 5, 'c75/r25': 6, 'c75/r75': 7, 'c75/s25': 8, 

100%|██████████| 424/424 [06:11<00:00,  1.14it/s]


Applying baseline correction (mode: zscore)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\jz421\AppData\Local\Temp\ipykernel_8472\999434550.py:92: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=2 parameter will result in a sampling frequency of 1024.0 Hz, which can cause aliasing artifacts.
  HG_base.decimate(2)
C:\Users\jz421\AppData\Local\Temp\ipykernel_8472\999434550.py:93: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=2 parameter will result in a sampling frequency of 1024.0 Hz, which can cause aliasing artifacts.
  resp.decimate(2)
C:\Users\jz421\AppData\Local\Temp\ipykernel_8472\999434550.py:94: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=2 parameter will result in a sampling frequency of 1024.0 Hz, which can cause aliasing artifacts.
  stim.decimate(2)
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:  1.0min
[Parallel(n_job

Saved significant channels for subject D0069 and mask Response to C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0069\sig_chans_D0069_Response.json
Saved significant channels for subject D0069 and mask Stimulus to C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0069\sig_chans_D0069_Stimulus.json


### testing with baseline period of 1 sec before stimulus and 1.3 seconds after avg response

In [ ]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)



print(layout.derivatives)
print(layout.derivatives.keys())

# raw = raw_from_layout(layout, subject=sub,
#                         extension='.edf', preload=True)
subjects = layout.get(return_type="id", target="subject")
sub = 'D0059'

filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False) #get line-noise filtered data

save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)


# do filtering now
## Crop raw data to minimize processing time
good = crop_empty_data(filt)

## remove bad channels
good.info['bads'] = channel_outlier_marker(good, 3, 2)
good.drop_channels(good.info['bads'])
good.load_data()

# CAR
ch_type = filt.get_channel_types(only_data_chs=True)[0]
good.set_eeg_reference(ref_channels="average", ch_type=ch_type)


RTs, skipped = calculate_RTs(good)
avg_RT = np.median(RTs)
print(avg_RT)
# make stimulus baseline EpochsTFR
times=[-1.5, avg_RT+1.3+0.5] #this is for 0.5 sec of padding on each side

# make baseline 
# make stimulus baseline EpochsTFR
# times=[-1, 0.5]

trials = trial_ieeg(good, "Stimulus", times, preload=True)
outliers_to_nan(trials, outliers=10)
HG_base = gamma.extract(trials, copy=False, n_jobs=1)
crop_pad(HG_base, "0.5s")

data_dict = {}  # Create an empty dictionary

for event, t in zip(("Stimulus", "Response"), ((-1, 1.5), (-1, 1.5))):
    times = [None, None]
    times[0] = t[0] - 0.5
    times[1] = t[1] + 0.5
    trials = trial_ieeg(good, event, times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_ev1 = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_ev1, "0.5s")

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore') #removed .average() part. Check with Aaron if this is okay.
    # HG_ev1.resample(100)
    # HG_ev1.filenames = good.filenames

    # Store the variable in the dictionary with the desired name
    data_dict[f"HG_ev1_{event}"] = HG_ev1
    data_dict[f"HG_ev1_{event}_rescaled"] = HG_ev1_rescaled

resp_evoke = data_dict["HG_ev1_Response"].average(method=lambda x: np.nanmean(x, axis=0))
resp_evoke_rescaled = data_dict["HG_ev1_Response_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))
stim_evoke = data_dict["HG_ev1_Stimulus"].average(method=lambda x: np.nanmean(x, axis=0))
stim_evoke_rescaled = data_dict["HG_ev1_Stimulus_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))

RTs, skipped = calculate_RTs(good)
avg_RT = np.median(RTs)

# Plot the evoked data
fig = resp_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))

fig.savefig(save_dir + '_HG_ev1_Response_fullTrialBaseline_zscore.png')

fig = stim_evoke_rescaled.plot(unit=False, scalings=dict(sEEG=1))
# Add a vertical line indicating the average reaction time to the plot
for ax in fig.axes:
    ax.axvline(x=avg_RT, color='r', linestyle='--')
    ax.axvline(x=-0.5, color='g', linestyle='--')
fig.savefig(save_dir + '_HG_ev1_Stimulus_fullTrialBaseline_zscore.png')

# # decimate for stats
# resp = data_dict["HG_ev1_Response"] #i think it shouldn't be the rescaled data because that is already divided by baseline, right? So it shouldn't be significantly different than baseline.
# stim = data_dict["HG_ev1_Stimulus"]
# HG_base.decimate(2)
# resp.decimate(2)
# stim.decimate(2)

# # import scipy
# resp_mask = stats.time_perm_cluster(resp._data, HG_base._data, 0.05, axis=0,
#                             n_perm=1000, n_jobs=6, ignore_adjacency=1)
# stim_mask = stats.time_perm_cluster(stim._data, HG_base._data, 0.05, axis=0,
#                             n_perm=1000, n_jobs=6, ignore_adjacency=1)

# # plot stats
# plt.figure(figsize=(16, 8))  # Adjust the width and height as needed
# plt.imshow(stim_mask, aspect='auto')
# # hold on from matlab
# # plt.imshow(x) #plot multiple things like this

# relabel_axes(0, 2500, -1000, 1500)
# plt.savefig(save_dir + '_stimulus_stats_big.png', dpi=300)

# plt.figure(figsize=(16, 8))  # Adjust the width and height as needed
# plt.imshow(resp_mask, aspect='auto')
# relabel_axes(0, 2500, -1000, 1500) 
# plt.savefig(save_dir + '_response_stats_big.png', dpi=300)

### testing with baseline as average of all data

In [ ]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)



print(layout.derivatives)
print(layout.derivatives.keys())

# raw = raw_from_layout(layout, subject=sub,
#                         extension='.edf', preload=True)
subjects = layout.get(return_type="id", target="subject")
sub = 'D0059'

filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False) #get line-noise filtered data

save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)


# do filtering now
## Crop raw data to minimize processing time
good = crop_empty_data(filt)

## remove bad channels
good.info['bads'] = channel_outlier_marker(good, 3, 2)
good.drop_channels(good.info['bads'])
good.load_data()

# CAR
ch_type = filt.get_channel_types(only_data_chs=True)[0]
good.set_eeg_reference(ref_channels="average", ch_type=ch_type)


# make baseline as average of all data
good_avg = good._data.mean()


# make stimulus baseline EpochsTFR
times=[-1, 0.5]

trials = trial_ieeg(good, "Stimulus", times, preload=True)
outliers_to_nan(trials, outliers=10)
HG_base = gamma.extract(trials, copy=False, n_jobs=1)
crop_pad(HG_base, "0.5s")

data_dict = {}  # Create an empty dictionary

for event, t in zip(("Stimulus", "Response"), ((-1, 1.5), (-1, 1.5))):
    times = [None, None]
    times[0] = t[0] - 0.5
    times[1] = t[1] + 0.5
    trials = trial_ieeg(good, event, times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_ev1 = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_ev1, "0.5s")
    HG_ev1_rescaled = HG_ev1 #redundant perhaps.
    HG_ev1_rescaled._data = HG_ev1_rescaled._data / good_avg

    # Store the variable in the dictionary with the desired name
    data_dict[f"HG_ev1_{event}"] = HG_ev1
    data_dict[f"HG_ev1_{event}_rescaled"] = HG_ev1_rescaled

resp_evoke = data_dict["HG_ev1_Response"].average(method=lambda x: np.nanmean(x, axis=0))
resp_evoke_rescaled = data_dict["HG_ev1_Response_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))
stim_evoke = data_dict["HG_ev1_Stimulus"].average(method=lambda x: np.nanmean(x, axis=0))
stim_evoke_rescaled = data_dict["HG_ev1_Stimulus_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))

RTs, skipped = calculate_RTs(good)
avg_RT = np.median(RTs)

# Plot the evoked data
fig = resp_evoke_rescaled.plot()


fig.savefig(save_dir + '_HG_ev1_Response_allDataBaseline.png')

fig = stim_evoke_rescaled.plot()
# Add a vertical line indicating the average reaction time to the plot
for ax in fig.axes:
    ax.axvline(x=avg_RT, color='r', linestyle='--')

fig.savefig(save_dir + '_HG_ev1_Stimulus_allDataBaseline.png')


# decimate for stats
resp = data_dict["HG_ev1_Response"] #i think it shouldn't be the rescaled data because that is already divided by baseline, right? So it shouldn't be significantly different than baseline.
stim = data_dict["HG_ev1_Stimulus"]
HG_base.decimate(2)
resp.decimate(2)
stim.decimate(2)

# import scipy
resp_mask = stats.time_perm_cluster(resp._data, good._data, 0.05, axis=0,
                            n_perm=1000, n_jobs=6, ignore_adjacency=1)
stim_mask = stats.time_perm_cluster(stim._data, good._data, 0.05, axis=0,
                            n_perm=1000, n_jobs=6, ignore_adjacency=1)

# plot stats
plt.figure(figsize=(16, 8))  # Adjust the width and height as needed
plt.imshow(stim_mask, aspect='auto')
# hold on from matlab
# plt.imshow(x) #plot multiple things like this

relabel_axes(0, 2500, -1000, 1500)
plt.savefig(save_dir + '_stimulus_stats_allBase.png', dpi=300)

plt.figure(figsize=(16, 8))  # Adjust the width and height as needed
plt.imshow(resp_mask, aspect='auto')
relabel_axes(0, 2500, -1000, 1500)
plt.savefig(save_dir + '_response_stats_allBase.png', dpi=300)

In [127]:
import numpy as np
import math

# Get the shape of stim._data
stim_shape = stim._data.shape

# Calculate desired total size for the last two dimensions
desired_total_size_last_two_dims = stim_shape[1] * stim_shape[2]

# Calculate the number of samples for the first dimension
num_samples = current_size // desired_total_size_last_two_dims

# If current_size is not an exact multiple of desired_total_size_last_two_dims, add 1 to num_samples
if current_size % desired_total_size_last_two_dims != 0:
    num_samples += 1

# Correct the desired size
desired_size = (num_samples, stim_shape[1], stim_shape[2])

# Calculate current size and the difference
current_size = good._data.size
size_diff = desired_total_size_last_two_dims * num_samples - current_size

print(f"desired_total_size_last_two_dims: {desired_total_size_last_two_dims}")
print(f"num_samples: {num_samples}")
print(f"current_size: {current_size}")
print(f"size_diff: {size_diff}")


# Flatten the array first
flat_data = good._data.flatten()

if size_diff > 0:
    # If the current size is smaller than the desired size, pad with zeros
    padded = np.pad(flat_data, (0, size_diff))
elif size_diff < 0:
    # If the current size is larger than the desired size, truncate the array
    padded = flat_data[:desired_total_size_last_two_dims * num_samples]
else:
    # If the sizes match, no padding or truncating is necessary
    padded = flat_data

# Now reshape
good_reshaped = padded.reshape(desired_size)

desired_total_size_last_two_dims: 222894
num_samples: 2532
current_size: 564188214
size_diff: 179394


In [ ]:
test = stats.time_perm_cluster(stim._data, good_reshaped, 0.05, axis=0,
                            n_perm=1000, n_jobs=6, ignore_adjacency=1)